In [18]:
import numpy as np
from math import asinh, atan, sqrt, pi
from time import time 
import os

def format_time(_time):
    v = round(_time, 3)
    return f"{v//3600}h {(v % 3600)//60}m {round(v % 60, 3)}s"

# newell f
def f(p):
  #print(type(p))
  x, y, z = abs(p[0]), abs(p[1]), abs(p[2])
  return + y / 2.0 * (z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         + z / 2.0 * (y**2 - x**2) * asinh(z / (sqrt(x**2 + y**2) + eps)) \
         - x*y*z * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))       \
         + 1.0 / 6.0 * (2*x**2 - y**2 - z**2) * sqrt(x**2 + y**2 + z**2)

# newell g
def g(p):
  x, y, z = p[0], p[1], abs(p[2])
  return + x*y*z * asinh(z / (sqrt(x**2 + y**2) + eps))                         \
         + y / 6.0 * (3.0 * z**2 - y**2) * asinh(x / (sqrt(y**2 + z**2) + eps)) \
         + x / 6.0 * (3.0 * z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         - z**3 / 6.0 * atan(x*y / (z * sqrt(x**2 + y**2 + z**2) + eps))        \
         - z * y**2 / 2.0 * atan(x*z / (y * sqrt(x**2 + y**2 + z**2) + eps))    \
         - z * x**2 / 2.0 * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))    \
         - x*y * sqrt(x**2 + y**2 + z**2) / 3.0


# demag tensor setup
def set_n_demag(c, permute, func, n_demag, dx):
  it = np.nditer(n_demag[:,:,:,c], flags=['multi_index'], op_flags=['writeonly'])
  while not it.finished:
    value = 0.0
    for i in np.rollaxis(np.indices((2,)*6), 0, 7).reshape(64, 6):
      idx = list(map(lambda k: (it.multi_index[k] + n[k] - 1) % (2*n[k] - 1) - n[k] + 1, range(3)))
      value += (-1)**sum(i) * func(list(map(lambda j: (idx[j] + i[j] - i[j+3]) * dx[j], permute)))
    it[0] = - value / (4 * pi * np.prod(dx))
    it.iternext()

# demag tensor setup
def set_n_demag_test(c, permute, func, n_demag, dx):
  it = np.nditer(n_demag[:,:,:,c], flags=['multi_index'], op_flags=['writeonly'])
  while not it.finished:
    value = 0.0
    for i in np.rollaxis(np.indices((2,)*6), 0, 7).reshape(64, 6):
      idx = list(map(lambda k: (it.multi_index[k] + n[k] - 1) % (2*n[k] - 1) - n[k] + 1, range(3)))
      value += (-1)**sum(i) * func(list(map(lambda j: (idx[j] + i[j] - i[j+3]) * dx[j], permute)))
    it[0] = - value / (4 * pi * np.prod(dx))
    it.iternext()


def calculate_demag_tensor(n, dx, demag_tensor_file = "demag_tensor.npy"):
    print("Calculating the demagnetization tensor")
    n_demag = np.zeros([2*i-1 for i in n] + [6])
    for i, t in enumerate(((f,0,1,2),(g,0,1,2),(g,0,2,1),(f,1,2,0),(g,1,2,0),(f,2,0,1))):
        set_n_demag(i, t[1:], t[0], n_demag=n_demag, dx=dx)
    np.save(demag_tensor_file, n_demag)
    return n_demag


def calculate_demag_tensor_test(n, dx, demag_tensor_file = "demag_tensor.npy"):
    print("Calculating the demagnetization tensor")
    n_demag = np.zeros([2*i-1 for i in n] + [6])
    for i, t in enumerate(((f,0,1,2),(g,0,1,2),(g,0,2,1),(f,1,2,0),(g,1,2,0),(f,2,0,1))):
        print(i, t)
        set_n_demag_test(i, t[1:], t[0], n_demag=n_demag, dx=dx)

    print(n_demag.shape)
    np.save(demag_tensor_file, n_demag)
    return n_demag

In [35]:
start = time()
# a very small number
eps = 1.0000000001e-18
n     = (100, 25, 1)
dx    = (1e-9, 1e-9, 1e-9)
calculate_demag_tensor(n, dx, demag_tensor_file=os.path.join("demag_calcs", f"demag_tensor_({n[0]},{n[1]},{n[2]}).npy"))
time_taken = time() - start
print(f"Cells : {np.prod(n)} ; time taken : {format_time(time_taken)}")

Calculating the demagnetization tensor
Cells : 2500 ; time taken : 0.0h 1.0m 45.198s


In [36]:
start = time()
# a very small number
eps = 1.0000000001e-18
n     = (100, 50, 1)
dx    = (1e-9, 1e-9, 1e-9)
calculate_demag_tensor(n, dx, demag_tensor_file=os.path.join("demag_calcs", f"demag_tensor_({n[0]},{n[1]},{n[2]}).npy"))
time_taken = time() - start
print(f"Cells : {np.prod(n)} ; time taken : {format_time(time_taken)}")

Calculating the demagnetization tensor
Cells : 5000 ; time taken : 0.0h 3.0m 16.512s


In [37]:
start = time()
# a very small number
eps = 1.0000000001e-18
n     = (100, 100, 1)
dx    = (1e-9, 1e-9, 1e-9)
calculate_demag_tensor(n, dx, demag_tensor_file=os.path.join("demag_calcs", f"demag_tensor_({n[0]},{n[1]},{n[2]}).npy"))
time_taken = time() - start
print(f"Cells : {np.prod(n)} ; time taken : {format_time(time_taken)}")

Calculating the demagnetization tensor
Cells : 10000 ; time taken : 0.0h 6.0m 38.719s


In [38]:
start = time()
# a very small number
eps = 1.0000000001e-18
n     = (100, 150, 1)
dx    = (1e-9, 1e-9, 1e-9)
calculate_demag_tensor(n, dx, demag_tensor_file=os.path.join("demag_calcs", f"demag_tensor_({n[0]},{n[1]},{n[2]}).npy"))
time_taken = time() - start
print(f"Cells : {np.prod(n)} ; time taken : {format_time(time_taken)}")

Calculating the demagnetization tensor
Cells : 15000 ; time taken : 0.0h 10.0m 11.344s


In [39]:
start = time()
# a very small number
eps = 1.0000000001e-18
n     = (100, 200, 1)
dx    = (1e-9, 1e-9, 1e-9)
calculate_demag_tensor(n, dx, demag_tensor_file=os.path.join("demag_calcs", f"demag_tensor_({n[0]},{n[1]},{n[2]}).npy"))
time_taken = time() - start
print(f"Cells : {np.prod(n)} ; time taken : {format_time(time_taken)}")

Calculating the demagnetization tensor
Cells : 20000 ; time taken : 0.0h 13.0m 30.475s


In [41]:
start = time()
# a very small number
eps = 1.00000000001e-18
n     = (100, 400, 1)
dx    = (1e-9, 1e-9, 1e-9)
calculate_demag_tensor(n, dx, demag_tensor_file=os.path.join("demag_calcs", f"demag_tensor_({n[0]},{n[1]},{n[2]}).npy"))
time_taken = time() - start
print(f"Cells : {np.prod(n)} ; time taken : {format_time(time_taken)}")

Calculating the demagnetization tensor
Cells : 40000 ; time taken : 0.0h 27.0m 9.043s
